<a href="https://colab.research.google.com/github/jewoolee0502/ComputerVision/blob/main/ECSE415_A4_Group50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECSE 415: Introduction to Computer Vision
###### Jewoo Lee - 260910789
###### Anthony Bonta -

## Assignment 4: Object Recognition

### Library Requirements

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Path

In [1]:
path_jay = '/content/drive/MyDrive/McGill/2025/Fall 2025/ECSE 415/A3/'
path_anthony = ''

### 1. Data Preparation

#### 1.1 Explore the Dataset

#### 1.2 Preprocess the Data

### 2. Baseline Methods

#### 2.1 Implementation of Baseline Object Recognition

#### 2.2 Analysis of Baseline Results

### 3. Model Implementation

#### 3.1 Choose a Model

#### 3.2 Data Augmentation

#### 3.3 Train the Model

### 4. Model Evaluation

#### 4.1 Evaluate Performance

#### 4.2 Visualize Results

#### 4.3 Generalization Test

### 5.  Prediction & Kaggle Competition